In [10]:
from pprint import pprint
from src.resolve_init import GetResolve
resolve = GetResolve()
project_manager = resolve.GetProjectManager()
project = project_manager.GetCurrentProject()
media_storage = resolve.GetMediaStorage()
media_pool = project.GetMediaPool()
root_folder = media_pool.GetRootFolder()
current_timeline = project.GetCurrentTimeline()

for clip in root_folder.GetClipList():
    pprint(clip.GetClipColor())

'Orange'
'Apricot'
'Yellow'
'Lime'
'Olive'
'Green'
'Teal'
'Navy'
'Blue'
'Purple'
'Violet'
'Pink'
'Tan'
'Beige'
'Brown'
'Chocolate'
